# School timetabling with Timefold in a Kotlin Notebook

*With Teacher class variant*

This Kotlin Jupyter Notebook solves a school timetabling problem with [Timefold](https://timefold.ai), the open source planning solver AI.
It optimizes the school schedule for students and teachers.

![School timetabling input output](https://timefold.ai/docs/timefold-solver/latest/_images/quickstart/school-timetabling/schoolTimetablingInputOutput.png)

This Notebook automatically assigns each lesson to a timeslot and a room, while adhering to hard and soft constraints: 

* A room can have at most one lesson at the same time.
* A teacher can teach at most one lesson at the same time.
* A student can attend at most one lesson at the same time.
* A teacher prefers to teach all lessons in the same room.
* A teacher prefers to teach sequential lessons and dislikes gaps between lessons.
* A student dislikes sequential lessons on the same subject.

## Dependencies

Add the Timefold solver dependency:

In [1]:
@file:DependsOn("ai.timefold.solver:timefold-solver-core:1.10.0")


## Domain

Create the domain data classes:

### Room

The `Room` class represents a location where lessons are taught, for example, `Room A` or `Room B`. For simplicity’s sake, all rooms are without capacity limits and they can accommodate all lessons.

In [2]:
data class Room(
    val name: String
)

### Timeslot

The `Timeslot` class represents a time interval when lessons are taught, for example, `Monday 10:30 - 11:30` or `Tuesday 13:30 - 14:30`. For simplicity’s sake, all time slots have the same duration and there are no time slots during lunch or other breaks.

In [3]:
import java.time.DayOfWeek
import java.time.LocalTime

data class Timeslot(
    val dayOfWeek: DayOfWeek,
    val startTime: LocalTime,
    val endTime: LocalTime
)

### Teacher

The `Teacher` class represents a teacher, for example `A. Turing` or `M. Curie`.

In [4]:
data class Teacher(
    val name: String
) {
    override fun toString(): String = "$name"
}

### Lesson

During a lesson, represented by the `Lesson` class, a teacher teaches a subject to a group of students, for example, `Math` by `A.Turing` for `9th grade` or `Chemistry` by `M.Curie` for `10th grade`. If a subject is taught multiple times per week by the same teacher to the same student group, there are multiple `Lesson` instances that are only distinguishable by `id`. For example, the 9th grade has six math lessons a week.

During solving, Timefold Solver changes the `timeslot` and `room` fields of the `Lesson` class, to assign each lesson to a time slot and a room. Therefore, those fields are annnoted with `@PlanningVariable` and the class with `@PlanningEntity`:

In [6]:
import ai.timefold.solver.core.api.domain.entity.PlanningEntity
import ai.timefold.solver.core.api.domain.lookup.PlanningId
import ai.timefold.solver.core.api.domain.variable.PlanningVariable

@PlanningEntity
data class Lesson(

    @PlanningId
    val id: Long,
    val subject: String,
    val teacher: Teacher,
    val studentGroup: String) {

    @PlanningVariable
    var timeslot: Timeslot? = null
    @PlanningVariable
    var room: Room? = null

    // No-arg constructor required for Timefold
    constructor() : this(0, "", Teacher(""), "")

    override fun toString(): String = "$subject"

}

## Constraints

The solver takes into account hard and soft constraints:

In [7]:
import ai.timefold.solver.core.api.score.buildin.hardsoft.HardSoftScore
import ai.timefold.solver.core.api.score.stream.Constraint
import ai.timefold.solver.core.api.score.stream.ConstraintFactory
import ai.timefold.solver.core.api.score.stream.ConstraintProvider
import ai.timefold.solver.core.api.score.stream.Joiners
import java.time.Duration

class TimeTableConstraintProvider : ConstraintProvider {

    override fun defineConstraints(constraintFactory: ConstraintFactory): Array<Constraint>? {
        return arrayOf(
            // Hard constraints
            roomConflict(constraintFactory),
            teacherConflict(constraintFactory),
            studentGroupConflict(constraintFactory),
            // Soft constraints
            teacherRoomStability(constraintFactory),
            teacherTimeEfficiency(constraintFactory),
            studentGroupSubjectVariety(constraintFactory)
        )
    }

    fun roomConflict(constraintFactory: ConstraintFactory): Constraint {
        // A room can accommodate at most one lesson at the same time.
        return constraintFactory
            // Select each pair of 2 different lessons ...
            .forEachUniquePair(
                Lesson::class.java,
                // ... in the same timeslot ...
                Joiners.equal(Lesson::timeslot),
                // ... in the same room ...
                Joiners.equal(Lesson::room)
            )
            // ... and penalize each pair with a hard weight.
            .penalize(HardSoftScore.ONE_HARD)
            .asConstraint("school-timetabling", "Room conflict");
    }

    fun teacherConflict(constraintFactory: ConstraintFactory): Constraint {
        // A teacher can teach at most one lesson at the same time.
        return constraintFactory
            .forEachUniquePair(
                Lesson::class.java,
                Joiners.equal(Lesson::timeslot),
                Joiners.equal(Lesson::teacher)
            )
            .penalize(HardSoftScore.ONE_HARD)
            .asConstraint("school-timetabling", "Teacher conflict");
    }

    fun studentGroupConflict(constraintFactory: ConstraintFactory): Constraint {
        // A student can attend at most one lesson at the same time.
        return constraintFactory
            .forEachUniquePair(
                Lesson::class.java,
                Joiners.equal(Lesson::timeslot),
                Joiners.equal(Lesson::studentGroup)
            )
            .penalize(HardSoftScore.ONE_HARD)
            .asConstraint("school-timetabling", "Student group conflict");
    }

    fun teacherRoomStability(constraintFactory: ConstraintFactory): Constraint {
        // A teacher prefers to teach in a single room.
        return constraintFactory
            .forEachUniquePair(
                Lesson::class.java,
                Joiners.equal(Lesson::teacher)
            )
            .filter { lesson1: Lesson, lesson2: Lesson -> lesson1.room !== lesson2.room }
            .penalize(HardSoftScore.ONE_SOFT)
            .asConstraint("school-timetabling", "Teacher room stability");
    }

    fun teacherTimeEfficiency(constraintFactory: ConstraintFactory): Constraint {
        // A teacher prefers to teach sequential lessons and dislikes gaps between lessons.
        return constraintFactory
            .forEach(Lesson::class.java)
            .join(Lesson::class.java,
                Joiners.equal(Lesson::teacher),
                Joiners.equal { lesson: Lesson -> lesson.timeslot?.dayOfWeek })
            .filter { lesson1: Lesson, lesson2: Lesson ->
                val between = Duration.between(
                    lesson1.timeslot?.endTime,
                    lesson2.timeslot?.startTime
                )
                !between.isNegative && between.compareTo(Duration.ofMinutes(30)) <= 0
            }
            .reward(HardSoftScore.ONE_SOFT)
            .asConstraint("school-timetabling", "Teacher time efficiency");
    }

    fun studentGroupSubjectVariety(constraintFactory: ConstraintFactory): Constraint {
        // A student group dislikes sequential lessons on the same subject.
        return constraintFactory
            .forEach(Lesson::class.java)
            .join(Lesson::class.java,
                Joiners.equal(Lesson::subject),
                Joiners.equal(Lesson::studentGroup),
                Joiners.equal { lesson: Lesson -> lesson.timeslot?.dayOfWeek })
            .filter { lesson1: Lesson, lesson2: Lesson ->
                val between = Duration.between(
                    lesson1.timeslot?.endTime,
                        lesson2.timeslot?.startTime
                )
                !between.isNegative && between.compareTo(Duration.ofMinutes(30)) <= 0
            }
            .penalize(HardSoftScore.ONE_SOFT)
            .asConstraint("school-timetabling", "Student group subject variety");
    }

}

### TimeTable

The `TimeTable` class represents a single dataset. It wraps all `Timeslot`, `Room`, and `Lesson` instances. Furthermore, because it contains all lessons, each with a specific planning variable state, it is a planning solution, and it has a `score` field:

In [8]:
import ai.timefold.solver.core.api.domain.solution.PlanningEntityCollectionProperty
import ai.timefold.solver.core.api.domain.solution.PlanningScore
import ai.timefold.solver.core.api.domain.solution.PlanningSolution
import ai.timefold.solver.core.api.domain.solution.ProblemFactCollectionProperty
import ai.timefold.solver.core.api.domain.valuerange.ValueRangeProvider
import ai.timefold.solver.core.api.score.buildin.hardsoft.HardSoftScore

@PlanningSolution
data class TimeTable(

    @ProblemFactCollectionProperty
    @ValueRangeProvider
    val timeslots: List<Timeslot>,
    @ProblemFactCollectionProperty
    @ValueRangeProvider
    val rooms: List<Room>,
    @PlanningEntityCollectionProperty
    val lessons: List<Lesson>) {

    @PlanningScore
    var score: HardSoftScore? = null

    // No-arg constructor required for Timefold
    constructor() : this(emptyList(), emptyList(), emptyList())

}

## Data generator

Generate some data for a small school timetable:

In [9]:
fun generateDemoData(): TimeTable {
    val timeslots: MutableList<Timeslot> = mutableListOf(
            Timeslot(DayOfWeek.MONDAY, LocalTime.of(8, 30), LocalTime.of(9, 30)),
            Timeslot(DayOfWeek.MONDAY, LocalTime.of(9, 30), LocalTime.of(10, 30)),
            Timeslot(DayOfWeek.MONDAY, LocalTime.of(10, 30), LocalTime.of(11, 30)),
            Timeslot(DayOfWeek.MONDAY, LocalTime.of(13, 30), LocalTime.of(14, 30)),
            Timeslot(DayOfWeek.MONDAY, LocalTime.of(14, 30), LocalTime.of(15, 30)),
    
            Timeslot(DayOfWeek.TUESDAY, LocalTime.of(8, 30), LocalTime.of(9, 30)),
            Timeslot(DayOfWeek.TUESDAY, LocalTime.of(9, 30), LocalTime.of(10, 30)),
            Timeslot(DayOfWeek.TUESDAY, LocalTime.of(10, 30), LocalTime.of(11, 30)),
            Timeslot(DayOfWeek.TUESDAY, LocalTime.of(13, 30), LocalTime.of(14, 30)),
            Timeslot(DayOfWeek.TUESDAY, LocalTime.of(14, 30), LocalTime.of(15, 30)))
    
    val rooms: MutableList<Room> = mutableListOf(
            Room("Room A"),
            Room("Room B"),
            Room("Room C"))
    
    val turing = Teacher("A. Turing")
    val curie = Teacher("M. Curie")
    val darwin = Teacher("C. Darwin")
    val jones = Teacher("I. Jones")
    val cruz = Teacher("P. Cruz")
    var nextId: Long = 0
    val lessons: MutableList<Lesson> = mutableListOf(
        Lesson(nextId++, "Math", turing, "9th grade"),
        Lesson(nextId++, "Math", turing, "9th grade"),
        Lesson(nextId++, "Physics", curie, "9th grade"),
        Lesson(nextId++, "Chemistry", curie, "9th grade"),
        Lesson(nextId++, "Biology", darwin, "9th grade"),
        Lesson(nextId++, "History", jones, "9th grade"),
        Lesson(nextId++, "English", jones, "9th grade"),
        Lesson(nextId++, "English", jones, "9th grade"),
        Lesson(nextId++, "Spanish", cruz, "9th grade"),
        Lesson(nextId++, "Spanish", cruz, "9th grade"),
        Lesson(nextId++, "Math", turing, "10th grade"),
        Lesson(nextId++, "Math", turing, "10th grade"),
        Lesson(nextId++, "Math", turing, "10th grade"),
        Lesson(nextId++, "Physics", curie, "10th grade"),
        Lesson(nextId++, "Chemistry", curie, "10th grade"),
        Lesson(nextId++, "French", curie, "10th grade"),
        Lesson(nextId++, "Geography", darwin, "10th grade"),
        Lesson(nextId++, "History", jones, "10th grade"),
        Lesson(nextId++, "English", cruz, "10th grade"),
        Lesson(nextId++, "Spanish", cruz, "10th grade"))
    return TimeTable(timeslots, rooms, lessons)
}

Line_9.jupyter.kts (46:16 - 24) The value changed at 'nextId++' is never used
java.lang.InterruptedException

## Solve it

Configure and run the solver:

In [10]:
import ai.timefold.solver.core.config.solver.SolverConfig
import ai.timefold.solver.core.api.solver.SolverFactory
import ai.timefold.solver.core.api.solver.Solver

val solverFactory: SolverFactory<TimeTable> = SolverFactory.create(SolverConfig()
        .withSolutionClass(TimeTable::class.java)
        .withEntityClasses(Lesson::class.java)
        .withConstraintProviderClass(TimeTableConstraintProvider::class.java)
        // The solver runs only for 5 seconds on this small dataset.
        // It's recommended to run for at least 5 minutes ("5m") otherwise.
        .withTerminationSpentLimit(Duration.ofSeconds(5)))

val problem: TimeTable = generateDemoData()

println("Solving the problem ...")
val solver: Solver<TimeTable> = solverFactory.buildSolver()
val solution: TimeTable = solver.solve(problem)
println("Solving finished with score (${solution.score}).")

Backend Internal error: Exception during psi2ir
File being compiled: (13,26) in Line_10.jupyter.kts
The root cause java.lang.IllegalStateException was thrown at: org.jetbrains.kotlin.ir.util.DescriptorSymbolTableExtension.referenceValueParameter(DescriptorSymbolTableExtension.kt:404)
Undefined parameter referenced: class Line_9_jupyter::this
owner=PROPERTY name:problem visibility:public modality:FINAL [val]; []
owner=SCRIPT; [class Line_4_jupyter::this, class Line_7_jupyter::this, class Line_2_jupyter::this, class Line_0_jupyter::this, class Line_1_jupyter::this, class Line_8_jupyter::this, class Line_3_jupyter::this, class Line_10_jupyter::this, class Line_5_jupyter::this, class Line_6_jupyter::this]
: KtCallExpression:
generateDemoData()

## Visualize the schedule

Show the timetable:

In [11]:
val lessonByRoomMap = solution.lessons.groupBy { lesson -> Pair(lesson.timeslot, lesson.room) }
val lessonByTeacherMap = solution.lessons.groupBy { lesson -> Pair(lesson.timeslot, lesson.teacher) }
val lessonByStudentGroupMap = solution.lessons.groupBy { lesson -> Pair(lesson.timeslot, lesson.studentGroup) }
HTML(buildString {
    append("<p style='font-size: x-large'>Score: ${solution.score}</p>")
    append("<p><b>By room</b></p>")
    append("<table><tr><th/>")
    for (room in solution.rooms) {
        append("<th>${room.name}</th>")
    }
    append("</tr>")
    for (timeslot in solution.timeslots) {
        append("<tr><th>${timeslot.dayOfWeek} ${timeslot.startTime} - ${timeslot.endTime}</th>")
        for (room in solution.rooms) {
            val cellLessons = lessonByRoomMap.get(Pair(timeslot, room)) ?: emptyList()
            append("<td>")
            append(cellLessons.map { it.subject }.joinToString(", "))
            append("</td>")
        }
        append("</tr>")
    }
    append("</table>")
    append("<p><b>By teacher</b></p>")
    append("<table><tr><th/>")
    val teachers = solution.lessons.map { it.teacher }.distinct()
    for (teacher in teachers) {
        append("<th>${teacher}</th>")
    }
    append("</tr>")
    for (timeslot in solution.timeslots) {
        append("<tr><th>${timeslot.dayOfWeek} ${timeslot.startTime} - ${timeslot.endTime}</th>")
        for (teacher in teachers) {
            val cellLessons = lessonByTeacherMap.get(Pair(timeslot, teacher)) ?: emptyList()
            append("<td>")
            append(cellLessons.map { it.subject }.joinToString(", "))
            append("</td>")
        }
        append("</tr>")
    }
    append("</table>")
    append("<p><b>By student group</b></p>")
    append("<table><tr><th/>")
    val studentGroups = solution.lessons.map { it.studentGroup }.distinct()
    for (studentGroup in studentGroups) {
        append("<th>${studentGroup}</th>")
    }
    append("</tr>")
    for (timeslot in solution.timeslots) {
        append("<tr><th>${timeslot.dayOfWeek} ${timeslot.startTime} - ${timeslot.endTime}</th>")
        for (studentGroup in studentGroups) {
            val cellLessons = lessonByStudentGroupMap.get(Pair(timeslot, studentGroup)) ?: emptyList()
            append("<td>")
            append(cellLessons.map { it.subject }.joinToString(", "))
            append("</td>")
        }
        append("</tr>")
    }
    append("</table>")

    val unassignedLessons = lessonByRoomMap.get(Pair(null, null))
    if (unassignedLessons != null && unassignedLessons.isNotEmpty()) {
        append("<p>Unassigned lessons</p>")
        append("<ul>")
        for (lesson in unassignedLessons) {
            append("<li>${lesson.subject} - ${lesson.teacher} - ${lesson.studentGroup}</li>")
        }
        append("</ul>")
    }
})

Backend Internal error: Exception during psi2ir
File being compiled: Line_11.jupyter.kts
The root cause java.lang.IllegalStateException was thrown at: org.jetbrains.kotlin.ir.util.SymbolTableSlice$Scoped.introduceLocal(SymbolTableSlice.kt:208)

## Statistics

For a big dataset, a schedule visualization is often too verbose.
Let's visualize the solution through statistics:

In [12]:
%use kandy

The problem is found in one of the loaded libraries: check library imports, dependencies and repositories
Error compiling code:
@file:Repository("https://packages.jetbrains.team/maven/p/kds/kotlin-ds-maven")
@file:DependsOn("org.jetbrains.kotlinx:kandy-lets-plot:0.6.0")
@file:DependsOn("org.jetbrains.kotlinx:kotlin-statistics-jvm:0.2.1")


Errors:
Backend Internal error: Exception during psi2ir
File being compiled: Line_12.jupyter.kts
The root cause java.lang.IllegalStateException was thrown at: org.jetbrains.kotlin.ir.util.SymbolTableSlice$Scoped.introduceLocal(SymbolTableSlice.kt:208)

org.jetbrains.kotlinx.jupyter.exceptions.ReplLibraryException: The problem is found in one of the loaded libraries: check library imports, dependencies and repositories
	at org.jetbrains.kotlinx.jupyter.exceptions.ReplLibraryExceptionKt.rethrowAsLibraryException(ReplLibraryException.kt:38)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl$ExecutionContext.doAddLibraries(CellExecutorImpl.kt:1

### Lessons per teacher per weekday

In [13]:
val teacherToLessonsMap = solution.lessons.groupBy {it.teacher}
val teachers = teacherToLessonsMap.keys.distinct()
val dayOfWeeks = enumValues<DayOfWeek>().toList()
val lessonCounts = dayOfWeeks.flatMap { dayOfWeek ->
    teachers.map { teacher ->
        (teacherToLessonsMap[teacher] ?: emptyList())
            .count { it.timeslot?.dayOfWeek == dayOfWeek }
    }
}

val dataset = mapOf(
    "Teacher" to List(dayOfWeeks.size) { teachers }.flatten().map { it.name },
    "Lessons" to lessonCounts,
    "Weekday" to dayOfWeeks.flatMap { dayOfWeek -> List(teachers.size) { dayOfWeek } }
)

dataset.plot {
    layout.title = "Lessons per teacher per weekday"
    groupBy("Weekday") {
        barsH {
            y("Teacher")
            x("Lessons")
            fillColor("Weekday") {
                scale = categoricalColorHue()
            }
            position = Position.stack()
        }
    }
}

Line_13.jupyter.kts (17:9 - 13) Unresolved reference: plot
Line_13.jupyter.kts (18:5 - 11) Unresolved reference: layout
Line_13.jupyter.kts (18:12 - 17) Variable expected
Line_13.jupyter.kts (19:5 - 12) Unresolved reference. None of the following candidates is applicable because of receiver type mismatch: 
public inline fun <T, K> Array<out TypeVariable(T)>.groupBy(keySelector: (TypeVariable(T)) -> TypeVariable(K)): Map<TypeVariable(K), List<TypeVariable(T)>> defined in kotlin.collections
public inline fun <T, K, V> Array<out TypeVariable(T)>.groupBy(keySelector: (TypeVariable(T)) -> TypeVariable(K), valueTransform: (TypeVariable(T)) -> TypeVariable(V)): Map<TypeVariable(K), List<TypeVariable(V)>> defined in kotlin.collections
public inline fun <K> BooleanArray.groupBy(keySelector: (Boolean) -> TypeVariable(K)): Map<TypeVariable(K), List<Boolean>> defined in kotlin.collections
public inline fun <K, V> BooleanArray.groupBy(keySelector: (Boolean) -> TypeVariable(K), valueTransform: (Bool

### Lessons per teacher per hour


In [14]:
val teacherToLessonsMap = solution.lessons.groupBy {it.teacher}
val teachers = teacherToLessonsMap.keys.distinct()
val startTimes = solution.timeslots.map { it.startTime.toString() }.distinct()
val lessonCounts = startTimes.flatMap { startTime ->
    teachers.map { teacher ->
        (teacherToLessonsMap[teacher] ?: emptyList())
            .count { it.timeslot?.startTime.toString() == startTime }
    }
}

val dataset = mapOf(
    "Teacher" to List(startTimes.size) { teachers }.flatten().map { it.name },
    "Lessons" to lessonCounts,
    "Start time" to startTimes.flatMap { startTime -> List(teachers.size) { startTime } }
)

dataset.plot {
    layout.title = "Lessons per teacher per hour"
    groupBy("Start time") {
        barsH {
            y("Teacher")
            x("Lessons")
            fillColor("Start time") {
                scale = categoricalColorHue()
            }
            position = Position.stack()
        }
    }
}

Line_14.jupyter.kts (17:9 - 13) Unresolved reference: plot
Line_14.jupyter.kts (18:5 - 11) Unresolved reference: layout
Line_14.jupyter.kts (18:12 - 17) Variable expected
Line_14.jupyter.kts (19:5 - 12) Unresolved reference. None of the following candidates is applicable because of receiver type mismatch: 
public inline fun <T, K> Array<out TypeVariable(T)>.groupBy(keySelector: (TypeVariable(T)) -> TypeVariable(K)): Map<TypeVariable(K), List<TypeVariable(T)>> defined in kotlin.collections
public inline fun <T, K, V> Array<out TypeVariable(T)>.groupBy(keySelector: (TypeVariable(T)) -> TypeVariable(K), valueTransform: (TypeVariable(T)) -> TypeVariable(V)): Map<TypeVariable(K), List<TypeVariable(V)>> defined in kotlin.collections
public inline fun <K> BooleanArray.groupBy(keySelector: (Boolean) -> TypeVariable(K)): Map<TypeVariable(K), List<Boolean>> defined in kotlin.collections
public inline fun <K, V> BooleanArray.groupBy(keySelector: (Boolean) -> TypeVariable(K), valueTransform: (Bool

## Analyze the score

Let's break down the score per constraint:

In [15]:
import ai.timefold.solver.core.api.solver.SolutionManager

val solutionManager = SolutionManager.create(solverFactory)
val scoreAnalysis = solutionManager.analyze(solution)

Backend Internal error: Exception during psi2ir
File being compiled: Line_15.jupyter.kts
The root cause java.lang.IllegalStateException was thrown at: org.jetbrains.kotlin.ir.util.SymbolTableSlice$Scoped.introduceLocal(SymbolTableSlice.kt:208)

And visualize it:

In [16]:
HTML(buildString {
    append("<p style='font-size: x-large'>Score: ${scoreAnalysis.score}</p>")
    append("<ul>")
    for (constraint in scoreAnalysis.constraintMap().values) {
        append("<li>${constraint.constraintRef().constraintName}: ${constraint.score.toShortString()}</li>")
    }
    append("</ul>")
})

Backend Internal error: Exception during psi2ir
File being compiled: Line_16.jupyter.kts
The root cause java.lang.IllegalStateException was thrown at: org.jetbrains.kotlin.ir.util.SymbolTableSlice$Scoped.introduceLocal(SymbolTableSlice.kt:208)

## Conclusion

To learn more about planning optimization, visit [timefold.ai](https://timefold.ai).